In [1]:
import glob
import os

log_files = glob.glob("*.log")
if log_files:
    file_path = log_files[0]  # 取列表的第一個檔案
    print(f"找到 log 檔案: {file_path}")
else:
    print("錯誤: 找不到 .log 檔案")
    file_path = None

folder = "./for_poisson/"
# 如果資料夾不存在，自動建立
os.makedirs(folder, exist_ok=True)
print(f"輸出資料夾: {folder}")

output_file = folder + "charges_output_2V.dat"  # 儲存的檔名

找到 log 檔案: 2V_tfsi_bmim_2cnt_CV.log
輸出資料夾: ./for_poisson/


In [2]:
import re

def search_for_charges(file_path, hist_file="hist_q_total_2V.dat"):
    """
    從 log 檔案中搜尋指定 iteration 範圍的電極電荷。
    
    改寫邏輯說明：
    1. 讀取 hist_q_total_V8.dat 的行數來確定分析的 frame 數量
    2. 自動從 log 檔案中抓取第一個和最後一個 iteration
    3. 根據 charge_density1D_V8.py 的邏輯（使用後半段 frame），
       計算對應的 iteration 範圍
    
    原理：
    - hist_q_total_V8.dat 的行數 = 分析的 frame 數量 (不完全對應 bin 數)
    - 但我們可以直接從 log 檔案抓取實際存在的 iteration
    """
    
    # Step 1: 讀取 log 檔案
    with open(file_path) as file:
        lines = file.readlines()
    
    # Step 2: 自動從 log 檔案中抓取第一個和最後一個 iteration
    # 使用正則表達式匹配 "Iteration X/Y" 格式
    iteration_pattern = re.compile(r'^Iteration (\d+)/(\d+)')
    
    first_iteration = None
    last_iteration = None
    total_iterations = None
    
    for line in lines:
        match = iteration_pattern.match(line)
        if match:
            current_iter = int(match.group(1))
            total_iterations = int(match.group(2))
            if first_iteration is None:
                first_iteration = current_iter
            last_iteration = current_iter
    
    if first_iteration is None or last_iteration is None:
        print("錯誤: 在 log 檔案中找不到 Iteration 標記")
        return None
    
    print(f"=== 自動偵測 Iteration 範圍 ===")
    print(f"  Log 檔案中的 Iteration 範圍: {first_iteration} ~ {last_iteration}")
    print(f"  總 Iteration 數: {total_iterations}")
    
    # Step 3: 根據 charge_density1D_V9.py 的邏輯計算後半段範圍
    # framestart = total_frames // 2
    mid_iteration = (first_iteration + last_iteration) // 2
    start_iteration = mid_iteration
    end_iteration = last_iteration
    
    print(f"  分析範圍 (後半段): {start_iteration} ~ {end_iteration}")
    print()
    
    # Step 4: 構建搜尋字串
    # 注意：原本的程式碼格式是 "X iteration"，這裡需要改成 "Iteration X/"
    first_iteration_index = 0
    last_iteration_index = 0
    charges_list = []
    
    c = f"Iteration {start_iteration}/"
    d = f"Iteration {end_iteration}/"
    
    # Step 5: 找到起始 iteration 的位置
    for a, line1 in enumerate(lines):
        if c in line1:
            first_iteration_index = a + 1
            print(f"起始位置: 行 {first_iteration_index}")
            print(f"  {line1.strip()}")
            break
    
    if first_iteration_index == 0:
        return f"未找到起始 iteration: {start_iteration}"
    
    # Step 6: 找到終止 iteration 的位置
    for b, line2 in enumerate(lines):
        if d in line2:
            last_iteration_index = b + 1
            print(f"終止位置: 行 {last_iteration_index}")
            print(f"  {line2.strip()}")
            break
    
    if last_iteration_index == 0:
        return f"未找到終止 iteration: {end_iteration}"
    
    # Step 7: 在範圍內搜尋電荷值
    for z in range(first_iteration_index, last_iteration_index):
        if "Q_numeric , Q_analytic charges on  anode" in lines[z]:
            charge_values = lines[z].strip().split()
            if len(charge_values) > 6:
                charges_list.append(charge_values[6])
    
    # Step 8: 輸出結果
    if charges_list:
        with open(output_file, "w") as f:
            for charge in charges_list:
                f.write(charge + "\n")
        
        print(f"\n已將 {len(charges_list)} 筆 charges 儲存至 {output_file}")
    else:
        print("未找到 charges")
    
    return charges_list

In [3]:
search_for_charges(file_path)

=== 自動偵測 Iteration 範圍 ===
  Log 檔案中的 Iteration 範圍: 1 ~ 10000
  總 Iteration 數: 10000
  分析範圍 (後半段): 5000 ~ 10000

起始位置: 行 519946
  Iteration 5000/10000
終止位置: 行 1039946
  Iteration 10000/10000

已將 250000 筆 charges 儲存至 ./for_poisson/charges_output_2V.dat


['-7.789868978857922',
 '-7.769490484311945',
 '-7.748057428549537',
 '-7.73327040883264',
 '-7.7256097414356715',
 '-7.7519006079215345',
 '-7.791139640275416',
 '-7.776302682666287',
 '-7.827093167206192',
 '-7.828384199624419',
 '-7.824346901894236',
 '-7.822717147254231',
 '-7.815592236891042',
 '-7.800579974475503',
 '-7.762980841819872',
 '-7.726437683840295',
 '-7.673647129693574',
 '-7.751827074514067',
 '-7.780376876084121',
 '-7.799314479325569',
 '-7.78675067487674',
 '-7.739710469541488',
 '-7.758843881668134',
 '-7.788621225846181',
 '-7.84139848336487',
 '-7.828832747018258',
 '-7.810988399179265',
 '-7.795731433261857',
 '-7.782155175184278',
 '-7.777453028975933',
 '-7.752773238072353',
 '-7.805508987226958',
 '-7.8243320886791',
 '-7.760333887044442',
 '-7.749722207632458',
 '-7.7426675376981375',
 '-7.7108632472547844',
 '-7.749648480200722',
 '-7.753844126765893',
 '-7.804264809906861',
 '-7.808803091759124',
 '-7.805545460686765',
 '-7.801809258235595',
 '-7.7579494